# Задание №6

### Требуется:

1) Отобрать записи, соответствующие странам с минимум 30 респонеднтами в выборке;

2) Получить сумму zero-crossing по каждой из записей;

3) Провести ANOVA-анализ по аттрибутам родного языка, пола и возраста для уровня значимости 0.15. Дискретность признака zero-crossing игнорировать.

In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import scipy.stats as st
import seaborn as sns
import pandas as pd
import librosa
import os
from tqdm import tqdm

Считаем данные о аудиозаписях:

In [3]:
data = pd.read_csv("speakers_all.csv")
data.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,24.0,12.0,"koussi, senegal",balanta,balanta,male,788,senegal,True,NaN,NaN,NaN
1,18.0,10.0,"buea, cameroon",cameroon,cameroon,male,1953,cameroon,True,NaN,NaN,NaN
2,48.0,8.0,"hong, adamawa, nigeria",fulfulde,fulfulde,male,1037,nigeria,True,NaN,NaN,NaN
3,42.0,42.0,"port-au-prince, haiti",haitian,haitian,male,1165,haiti,True,NaN,NaN,NaN
4,40.0,35.0,"port-au-prince, haiti",haitian,haitian,male,1166,haiti,True,NaN,NaN,NaN


Видим лишние колонки, избавимся от них. Оставим и переименуем один из столбцов для суммы zero-crossing. Также удалим все записи, где нет файла аудио (*file_missing? = True*).

In [4]:
data = data.drop(['Unnamed: 10', 'Unnamed: 11'], axis=1)
data = data[data['file_missing?'] == False]
data = data.drop(['file_missing?'], axis=1)

Отоберем записи, соответствующие странам с минимум 30 респонеднтами в выборке:

In [5]:
X = data.groupby('country').agg(['count'])
countries = X[X['speakerid']['count']>30].index
data_new = data[data.country.isin(countries)]
data_new.columns = ['age', 'age_onset', 'birthplace', 'filename', 'native_language', 'sex', 'speakerid', 'country', 'zcr']
data_new.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,zcr
50,20.0,5.0,"addis ababa, ethiopia",amharic1,amharic,female,6,ethiopia,NaN
51,29.0,19.0,"gonder, ethiopia",amharic10,amharic,female,998,ethiopia,NaN
52,24.0,17.0,"addis ababa, ethiopia",amharic11,amharic,female,1129,ethiopia,NaN
53,26.0,15.0,"addis ababa, ethiopia",amharic12,amharic,female,1130,ethiopia,NaN
54,28.0,6.0,"addis ababa, ethiopia",amharic13,amharic,female,1131,ethiopia,NaN


Разархивируем нужные файлы:

In [6]:
from tqdm import tqdm
from zipfile import ZipFile
list_of_audio = data_new.filename.values

with ZipFile('recordings.zip', 'r') as zipObj:
    for fileName in tqdm(list_of_audio):
        zipObj.extract('recordings/'+fileName+'.mp3')

100%|██████████| 1045/1045 [00:42<00:00, 24.40it/s]


Для Windows:

In [32]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/ffmpeg/bin/'

Получим сумму zero-crossing по каждой из записей и запишем в *zcr*:

In [ ]:
# Если этот шаг уже был выполнен, можно считать данные из файла with_zcr.csv
from tqdm import tqdm
for i in tqdm(data_new.index):
    x , sr = librosa.load('recordings/'+ data_new.loc[i, 'filename'] +'.mp3')
    zero_crossings = librosa.zero_crossings(x, pad=False)
    data_new.loc[i, 'zcr'] = sum(zero_crossings)

Удалим ненужные колонки:

In [41]:
data_new = data_new.drop(['age_onset', 'birthplace', 'filename', 'speakerid', 'country'], axis=1)
data_new.head()

,age,native_language,sex,zcr
50,20.0,amharic,female,67652.0
51,29.0,amharic,female,73628.0
52,24.0,amharic,female,80456.0
53,26.0,amharic,female,64468.0
54,28.0,amharic,female,103746.0


Сохраним данные:

In [42]:
data_new.to_csv('with_zcr.csv')

In [30]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
lm = ols('zcr ~ C(native_language) + C(age) + C(sex)',data=data_new).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(nl),55.0,2.626603e+11,4.775643e+09,6.649470,2.256077e-38
C(age),74.0,7.640657e+10,1.032521e+09,1.437653,1.121252e-02
C(sex),1.0,1.120876e+10,1.120876e+10,15.606759,8.397115e-05
Residual,915.0,6.571521e+11,7.181990e+08,NaN,NaN


Все признаки важны с p-value < 0.15<br>
'age' почти на границе, возможно нам не нужно 75 различных возрастов, попробуем их собрать в группы по 5 лет.

In [92]:
borders = [20,22.5,25,30,37.5,50]
for i in tqdm(data_new.index):
    if data_new.loc[i, 'age'] <= 20:
        data_new.loc[i, 'age'] = 0
    for j in range(len(borders)-1):
        if borders[j] < data_new.loc[i, 'age'] and data_new.loc[i, 'age'] <= borders[j+1]:
            data_new.loc[i, 'age'] = j+1
            
    if data_new.loc[i, 'age'] >= 50:
        data_new.loc[i, 'age'] = len(borders)+1

100%|██████████| 1045/1045 [00:00<00:00, 1793.68it/s]


In [93]:
X = data_new.groupby('age').agg(['count'])
X

,nl,sex,zcr
,count,count,count
age,,,
0.0,195,195,195
1.0,130,130,130
2.0,145,145,145
3.0,153,153,153
4.0,128,128,128
5.0,149,149,149
7.0,145,145,145


Группы примерно равны по мощности.

In [94]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
lm = ols('zcr ~ C(nl) + C(age) + C(sex)',data=data_new).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(nl),55.0,2.626603e+11,4.775643e+09,6.545327,4.337149e-38
C(age),6.0,1.669044e+10,2.781740e+09,3.812554,9.191561e-04
C(sex),1.0,1.103318e+10,1.103318e+10,15.121692,1.075786e-04
Residual,982.0,7.164930e+11,7.296263e+08,NaN,NaN


Важность возраста возрасла<br>
Посмотрим межфакторное взаимодействие признаков

In [96]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
lm = ols('zcr ~ C(nl) * C(age) * C(sex)',data=data_new).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(nl),55.0,2.626603e+11,4.775643e+09,7.056153,3.046009e-40
C(age),6.0,1.669044e+10,2.781740e+09,4.110103,4.503338e-04
C(sex),1.0,1.103318e+10,1.103318e+10,16.301856,5.919142e-05
C(nl):C(age),330.0,2.699684e+11,8.180862e+08,1.208746,1.841397e-02
C(nl):C(sex),55.0,4.296283e+10,7.811424e+08,1.154161,2.121422e-01
C(age):C(sex),6.0,1.244624e+09,2.074373e+08,0.306495,9.336884e-01
C(nl):C(age):C(sex),330.0,2.037038e+11,6.172841e+08,0.912056,8.353412e-01
Residual,802.0,5.427980e+11,6.768054e+08,NaN,NaN


Гипотеза об отсутствии межфакторного взаимодействия признаков не отклоняется для всех пар.

## Вывод: 
### На сумму *zero-crossing-rate* влияют все признаки по отдельности, хотя исходный 'age' с довольно высоким p-value. Отвергнуть гипотезу, что межфаркторного взаимодействия нет, мы не можем.